# Shuffling
- 그룹핑시 데이터를 한 노드에서 다른 노드로 옮길 때 발생, 성능을 (많이) 저하시킨다.
- 여러 네트워크 연산을 일으키기 때문에 네트워크 코스트가 크다.

### Shuffle을 일으킬 수 있는 작업들
- Join, leftOuterJoin, rightOuterJoin
- GroupByKey
- ReduceByKey
- CombineByKey
- Distinct
- Intersection
- Repartition
- Coalesce

### Shuffling은 언제 일어날까?
- 결과로 나오는 RDD가 원본 RDD의 다른 요소를 참조하거나 , 다른 RDD를 참조할 때

### Partitioner를 이용한 성능최적화 
- GroupByKeys + Reduce를 하게되면 => 성능을 매우 떨어짐


<table>
  <tr>
    <td>
      <img src="GroupByKey+Reduce.png" alt="Image 1" width="500" height="300">
    </td>
    <td>
      <img src="ReduceByKey.Png" alt="Image 2" width="500" height="300">
    </td>
  </tr>
</table>

### Shuffle을 최소화 하려면
- 미리 파티션을 만들어 두고 캐싱 후 reduceByKey 실행
- 미리 파티션을 만들어 두고 캐싱 후 Join 실행
- 둘다 파티션과 캐싱을 조합해서 최대한 로컬 환경에서 연산이 실행되도록 하는 방식

***셔플을 최소화해서 10배의 성능 향상이 가능하다***

# 2. 예제

In [5]:
from operator import add
from pyspark import SparkConf, SparkContext

import warnings 
warnings.simplefilter(action='ignore')

conf = SparkConf().setMaster('local').setAppName('category-review-average')
sc = SparkContext(conf=conf)

23/04/02 17:51:58 WARN Utils: Your hostname, Keemyoui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.79 instead (on interface en0)
23/04/02 17:51:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 17:51:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/02 17:51:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# reduceByKey
# 임의의 텍스트 데이터
text_data = ["Spark is a fast and general-purpose cluster-computing system",
             "It provides high-level APIs in Java, Scala, Python and R",
             "Spark is built on the Hadoop Distributed File System"]

# 데이터를 RDD로 변환
textRDD = sc.parallelize(text_data)

# 단어 빈도 계산
word_frequency = (textRDD
    .flatMap(lambda line: line.split())            # 각 줄을 단어로 분할
    .map(lambda word: (word.lower(), 1))           # 각 단어를 소문자로 변환하고 (word, 1) 쌍을 생성
    .reduceByKey(lambda a, b: a + b))              # 같은 단어의 값을 합산

# 결과 출력
for word, count in word_frequency.collect():
    print(f"{word}: {count}")

spark: 2
is: 2
a: 1
fast: 1
and: 2
general-purpose: 1
cluster-computing: 1
system: 2
it: 1
provides: 1
high-level: 1
apis: 1
in: 1
java,: 1
scala,: 1
python: 1
r: 1
built: 1
on: 1
the: 1
hadoop: 1
distributed: 1
file: 1


In [7]:
# 임의의 텍스트 데이터
text_data = ["Spark is a fast and general-purpose cluster-computing system",
             "It provides high-level APIs in Java, Scala, Python and R",
             "Spark is built on the Hadoop Distributed File System"]

# 데이터를 RDD로 변환
textRDD = sc.parallelize(text_data)

# 단어 빈도 계산
word_frequency = (textRDD
    .flatMap(lambda line: line.split())              # 각 줄을 단어로 분할
    .map(lambda word: (word.lower(), 1))             # 각 단어를 소문자로 변환하고 (word, 1) 쌍을 생성
    .groupByKey()                                    # 같은 단어를 그룹화
    .map(lambda wc: (wc[0], sum(wc[1]))))            # 그룹화된 단어의 값을 합산

# 결과 출력
for word, count in word_frequency.collect():
    print(f"{word}: {count}")

spark: 2
is: 2
a: 1
fast: 1
and: 2
general-purpose: 1
cluster-computing: 1
system: 2
it: 1
provides: 1
high-level: 1
apis: 1
in: 1
java,: 1
scala,: 1
python: 1
r: 1
built: 1
on: 1
the: 1
hadoop: 1
distributed: 1
file: 1


# Partition은 어떻게 결정될까?

### 데이터가 어느 노드 /파티션에 들어가는지는 어떻게 결정될까?

### 파티션의 목적
- 데이터를 최대한 균일하게 퍼트리고 쿼리가 같이 되는 데이터를 최대한 옆에 두어 검색 성능을 향상시키는 것
- 파티션은 PairedRDD일 때만 의미가 있음, 일반 RDD는 처음부터 끝까지 scanning을 해야함(의미없음)
- 해시로 파티셔닝했을 때 어떤 데이터를 찾고 싶다 했을 때 Key를 통해 바로 찾을 수 있음

### Partition의 특징
- RDD는 쪼개져서 여러 파티션에 저장됨
- 하나의 파티션은 하나의 노드 (서버)에 저장됨
- 하나의 노드는 여러개의 파티션을 가질 수 있음
- 파티션의 크기와 배치는 자유롭게 설정 가능하며 성능에 큰 영향을 미침
- Key-Value RDD를 사용할때만 의미가 있다.

***스파크의 파티셔닝 == 일반 프로그래밍에서 자료구조를 선택하는 것**

### Partition의 종류
- Hash Partitioning
- Range Partitioning